In [20]:
# Create a data directory
import os
os.makedirs("data", exist_ok=True)

# Download the GEO raw tar file
!wget -O data/GSE297365_RAW.tar \
ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE297nnn/GSE297365/suppl/GSE297365_RAW.tar

--2025-12-21 06:27:27--  ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE297nnn/GSE297365/suppl/GSE297365_RAW.tar
           => ‘data/GSE297365_RAW.tar’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.31, 130.14.250.7, 130.14.250.10, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.31|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /geo/series/GSE297nnn/GSE297365/suppl ... done.
==> SIZE GSE297365_RAW.tar ... 357468160
==> PASV ... done.    ==> RETR GSE297365_RAW.tar ... done.
Length: 357468160 (341M) (unauthoritative)

GSE297365_RAW.tar   100%[===================>] 340.91M  61.9MB/s    in 5.5s    

2025-12-21 06:27:34 (61.9 MB/s) - ‘data/GSE297365_RAW.tar’ saved [357468160]



In [21]:
# Extract the tar archive
!tar -xvf data/GSE297365_RAW.tar -C data/

GSM8989497_105_2g_ABT_263_raw_expression.h5
GSM8989498_105_2g_DMSO_raw_expression.h5
GSM8989499_HD_7HA_ABT_263_raw_expression.h5
GSM8989500_HD_7HA_DMSO_raw_expression.h5


In [22]:
!ls data/

GSE297365_RAW.tar
GSM8989497_105_2g_ABT_263_raw_expression.h5
GSM8989498_105_2g_DMSO_raw_expression.h5
GSM8989499_HD_7HA_ABT_263_raw_expression.h5
GSM8989500_HD_7HA_DMSO_raw_expression.h5


In [23]:
!find data -maxdepth 3

data
data/GSE297365_RAW.tar
data/GSM8989497_105_2g_ABT_263_raw_expression.h5
data/GSM8989500_HD_7HA_DMSO_raw_expression.h5
data/GSM8989498_105_2g_DMSO_raw_expression.h5
data/GSM8989499_HD_7HA_ABT_263_raw_expression.h5


In [24]:
!pip install scanpy anndata h5py

In [25]:
import h5py

file = "data/GSM8989497_105_2g_ABT_263_raw_expression.h5"

with h5py.File(file, "r") as f:
    print(list(f.keys()))

['matrix']


In [26]:
import scanpy as sc

file = "data/GSM8989497_105_2g_ABT_263_raw_expression.h5"

adata = sc.read_10x_h5(file)
adata

/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1808: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1808: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 1851517 × 60623
    var: 'gene_ids', 'feature_types', 'genome'

In [27]:
adata.obs.head()


""
AAACCCAAGAAACCCA-1
AAACCCAAGAAAGCGA-1
AAACCCAAGAAATCCA-1
AAACCCAAGAAATTGC-1
AAACCCAAGAACAGGA-1


In [28]:
adata.var.head(10)

,gene_ids,feature_types,genome
DDX11L1,ENSG00000223972,Gene Expression,refdata-cellranger-GRC38-3.1.0
WASH7P,ENSG00000227232,Gene Expression,refdata-cellranger-GRC38-3.1.0
MIR6859-1,ENSG00000278267,Gene Expression,refdata-cellranger-GRC38-3.1.0
MIR1302-2HG,ENSG00000243485,Gene Expression,refdata-cellranger-GRC38-3.1.0
MIR1302-2,ENSG00000284332,Gene Expression,refdata-cellranger-GRC38-3.1.0
FAM138A,ENSG00000237613,Gene Expression,refdata-cellranger-GRC38-3.1.0
OR4G4P,ENSG00000268020,Gene Expression,refdata-cellranger-GRC38-3.1.0
OR4G11P,ENSG00000240361,Gene Expression,refdata-cellranger-GRC38-3.1.0
OR4F5,ENSG00000186092,Gene Expression,refdata-cellranger-GRC38-3.1.0
AL627309.1,ENSG00000238009,Gene Expression,refdata-cellranger-GRC38-3.1.0


In [29]:
adata.var_names[:10]

Index(['DDX11L1', 'WASH7P', 'MIR6859-1', 'MIR1302-2HG', 'MIR1302-2', 'FAM138A',
       'OR4G4P', 'OR4G11P', 'OR4F5', 'AL627309.1'],
      dtype='object')

In [30]:
adata.var_names.duplicated().sum()

np.int64(1211)

In [31]:
adata.var_names_make_unique()

/tmp/ipython-input-287840421.py:1: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`. Example key collisions generated by the make_index_unique algorithm: ['SNORD116-1', 'SNORD116-2', 'SNORD116-3', 'SNORD116-4', 'SNORD116-5']
  adata.var_names_make_unique()


In [32]:
# QC metrics
adata.obs["n_counts"] = adata.X.sum(axis=1).A1
adata.obs["n_genes"] = (adata.X > 0).sum(axis=1).A1

adata.obs[["n_counts", "n_genes"]].describe()

,n_counts,n_genes
count,1.851517e+06,1.851517e+06
mean,7.255589e+01,2.919349e+01
std,1.019647e+03,2.858996e+02
min,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00
50%,1.000000e+00,1.000000e+00
75%,1.000000e+00,1.000000e+00
max,7.244500e+04,1.020200e+04


In [33]:
adata_filt = adata[
    (adata.obs["n_genes"] >= 500) &
    (adata.obs["n_counts"] >= 1000) &
    (adata.obs["n_genes"] <= 8000)
].copy()

adata_filt

AnnData object with n_obs × n_vars = 8987 × 60623
    obs: 'n_counts', 'n_genes'
    var: 'gene_ids', 'feature_types', 'genome'

In [34]:
adata_filt.obs["disease"] = "PMS"
adata_filt.obs["treatment"] = "ABT_263"
adata_filt.obs["donor"] = "105_2g"
adata_filt.obs["sample"] = "PMS_ABT"

In [35]:

adata_filt.write("data/PMS_ABT_filtered.h5ad")

In [36]:
!ls -lh data/

total 922M
-rw-r--r-- 1 root root 341M Dec 21 06:27 GSE297365_RAW.tar
-rw-r--r-- 1 root root  97M Jan 17  2025 GSM8989497_105_2g_ABT_263_raw_expression.h5
-rw-r--r-- 1 root root  54M Jan 17  2025 GSM8989498_105_2g_DMSO_raw_expression.h5
-rw-r--r-- 1 root root 106M Jan 17  2025 GSM8989499_HD_7HA_ABT_263_raw_expression.h5
-rw-r--r-- 1 root root  86M Jan 17  2025 GSM8989500_HD_7HA_DMSO_raw_expression.h5
-rw-r--r-- 1 root root 240M Dec 21 06:29 PMS_ABT_filtered.h5ad


In [37]:
import scanpy as sc

adata_check = sc.read_h5ad("data/PMS_ABT_filtered.h5ad")
adata_check

AnnData object with n_obs × n_vars = 8987 × 60623
    obs: 'n_counts', 'n_genes', 'disease', 'treatment', 'donor', 'sample'
    var: 'gene_ids', 'feature_types', 'genome'

In [38]:
# What metadata do we have?
adata_check.obs.head()

,n_counts,n_genes,disease,treatment,donor,sample
AAACCCAAGTGTAGTA-1,22374.0,6087,PMS,ABT_263,105_2g,PMS_ABT
AAACCCACAGTTACCA-1,13908.0,5216,PMS,ABT_263,105_2g,PMS_ABT
AAACCCACATTGAGCT-1,15994.0,4569,PMS,ABT_263,105_2g,PMS_ABT
AAACCCAGTGACACAG-1,11020.0,4117,PMS,ABT_263,105_2g,PMS_ABT
AAACCCAGTTCGGTAT-1,25389.0,6495,PMS,ABT_263,105_2g,PMS_ABT


In [39]:

# What gene annotations exist?
adata_check.var.head()

,gene_ids,feature_types,genome
DDX11L1,ENSG00000223972,Gene Expression,refdata-cellranger-GRC38-3.1.0
WASH7P,ENSG00000227232,Gene Expression,refdata-cellranger-GRC38-3.1.0
MIR6859-1,ENSG00000278267,Gene Expression,refdata-cellranger-GRC38-3.1.0
MIR1302-2HG,ENSG00000243485,Gene Expression,refdata-cellranger-GRC38-3.1.0
MIR1302-2,ENSG00000284332,Gene Expression,refdata-cellranger-GRC38-3.1.0


In [40]:
# Shape and sparsity
adata_check.n_obs, adata_check.n_vars

(8987, 60623)

In [41]:
# Top expressed genes (raw counts)
import numpy as np

gene_sums = np.array(adata_check.X.sum(axis=0)).flatten()
top_genes = adata_check.var_names[np.argsort(gene_sums)[-10:]]
top_genes

Index(['RPL13', 'RPS18', 'EEF1A1', 'RPS8', 'RPL37A', 'RPLP1', 'RPL41', 'RPS19',
       'MT-RNR2', 'MALAT1'],
      dtype='object')

In [42]:


del adata_check
import gc
gc.collect()

42

In [43]:
import scanpy as sc

file = "data/GSM8989498_105_2g_DMSO_raw_expression.h5"
adata = sc.read_10x_h5(file)
adata

/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1808: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1808: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 1764145 × 60623
    var: 'gene_ids', 'feature_types', 'genome'

In [44]:
adata.var_names_make_unique()

/tmp/ipython-input-287840421.py:1: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`. Example key collisions generated by the make_index_unique algorithm: ['SNORD116-1', 'SNORD116-2', 'SNORD116-3', 'SNORD116-4', 'SNORD116-5']
  adata.var_names_make_unique()


In [45]:
# QC metrics
adata.obs["n_counts"] = adata.X.sum(axis=1).A1
adata.obs["n_genes"] = (adata.X > 0).sum(axis=1).A1

adata.obs[["n_counts", "n_genes"]].describe()

,n_counts,n_genes
count,1.764145e+06,1.764145e+06
mean,4.862700e+01,1.569357e+01
std,1.201703e+03,2.359746e+02
min,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00
50%,1.000000e+00,1.000000e+00
75%,2.000000e+00,2.000000e+00
max,1.437580e+05,1.357200e+04


In [46]:
adata_filt = adata[
    (adata.obs["n_genes"] >= 500) &
    (adata.obs["n_counts"] >= 1000) &
    (adata.obs["n_genes"] <= 8000)
].copy()

adata_filt

AnnData object with n_obs × n_vars = 3281 × 60623
    obs: 'n_counts', 'n_genes'
    var: 'gene_ids', 'feature_types', 'genome'

In [47]:
adata_filt.obs["disease"] = "PMS"
adata_filt.obs["treatment"] = "DMSO"
adata_filt.obs["donor"] = "105_2g"
adata_filt.obs["sample"] = "PMS_DMSO"

In [48]:
adata_filt.write("data/PMS_DMSO_filtered.h5ad")

In [49]:
del adata
del adata_filt

import gc
gc.collect()

31

In [50]:
import scanpy as sc

file = "data/GSM8989499_HD_7HA_ABT_263_raw_expression.h5"
adata = sc.read_10x_h5(file)
adata

/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1808: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1808: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 1948528 × 60623
    var: 'gene_ids', 'feature_types', 'genome'

In [51]:
adata.var_names_make_unique()

/tmp/ipython-input-287840421.py:1: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`. Example key collisions generated by the make_index_unique algorithm: ['SNORD116-1', 'SNORD116-2', 'SNORD116-3', 'SNORD116-4', 'SNORD116-5']
  adata.var_names_make_unique()


In [52]:
# QC metrics
adata.obs["n_counts"] = adata.X.sum(axis=1).A1
adata.obs["n_genes"] = (adata.X > 0).sum(axis=1).A1

adata.obs[["n_counts", "n_genes"]].describe()

,n_counts,n_genes
count,1.948528e+06,1.948528e+06
mean,6.788657e+01,2.978015e+01
std,8.434061e+02,2.514934e+02
min,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00
50%,1.000000e+00,1.000000e+00
75%,1.000000e+00,1.000000e+00
max,6.883700e+04,9.346000e+03


In [53]:


adata_filt = adata[
    (adata.obs["n_genes"] >= 500) &
    (adata.obs["n_counts"] >= 1000) &
    (adata.obs["n_genes"] <= 8000)
].copy()

adata_filt

AnnData object with n_obs × n_vars = 11424 × 60623
    obs: 'n_counts', 'n_genes'
    var: 'gene_ids', 'feature_types', 'genome'

In [54]:
adata_filt.obs["disease"] = "Healthy"
adata_filt.obs["treatment"] = "ABT_263"
adata_filt.obs["donor"] = "7HA"
adata_filt.obs["sample"] = "HD_ABT"

In [55]:
adata_filt.write("data/HD_ABT_filtered.h5ad")

In [56]:
del adata
del adata_filt

import gc
gc.collect()

31

In [57]:
import scanpy as sc

file = "data/GSM8989500_HD_7HA_DMSO_raw_expression.h5"
adata = sc.read_10x_h5(file)
adata

/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1808: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/usr/local/lib/python3.12/dist-packages/anndata/_core/anndata.py:1808: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


AnnData object with n_obs × n_vars = 1791147 × 60623
    var: 'gene_ids', 'feature_types', 'genome'

In [58]:
adata.var_names_make_unique()

/tmp/ipython-input-287840421.py:1: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`. Example key collisions generated by the make_index_unique algorithm: ['SNORD116-1', 'SNORD116-2', 'SNORD116-3', 'SNORD116-4', 'SNORD116-5']
  adata.var_names_make_unique()


In [59]:
# QC metrics
adata.obs["n_counts"] = adata.X.sum(axis=1).A1
adata.obs["n_genes"] = (adata.X > 0).sum(axis=1).A1

adata.obs[["n_counts", "n_genes"]].describe()

,n_counts,n_genes
count,1.791147e+06,1.791147e+06
mean,6.531795e+01,2.643504e+01
std,9.458843e+02,2.790752e+02
min,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00
50%,1.000000e+00,1.000000e+00
75%,1.000000e+00,1.000000e+00
max,6.499500e+04,9.889000e+03


In [60]:
adata_filt = adata[
    (adata.obs["n_genes"] >= 500) &
    (adata.obs["n_counts"] >= 1000) &
    (adata.obs["n_genes"] <= 8000)
].copy()

adata_filt

AnnData object with n_obs × n_vars = 9331 × 60623
    obs: 'n_counts', 'n_genes'
    var: 'gene_ids', 'feature_types', 'genome'

In [61]:
adata_filt.obs["disease"] = "Healthy"
adata_filt.obs["treatment"] = "DMSO"
adata_filt.obs["donor"] = "7HA"
adata_filt.obs["sample"] = "HD_DMSO"

In [62]:
adata_filt.write("data/HD_DMSO_filtered.h5ad")


In [63]:
del adata
del adata_filt

import gc
gc.collect()

31